In [42]:
import pandas as pd
from functools import reduce
from IPython.display import IFrame 

In [43]:
location ='https://raw.githubusercontent.com/Public-Policy-COVID/'
fileEmma = location+'Emma-li/main/emma_data.csv'
fileBryn = location + 'Bryn-Bandt-Law/main/bryn.bl.clean.data.csv'
fileAyushi = location + 'Ayushi-Dhar/main/ayushi_dhar.csv'
fileNick = location + 'Nick-Maue/main/westcoastrace.csv'
emma_data = pd.read_csv(fileEmma)
bryn_data = pd.read_csv(fileBryn)
ayushi_data = pd.read_csv(fileAyushi)
nick_data = pd.read_csv(fileNick)

In [46]:
merge1 = pd.merge(left=ayushi_data, right=emma_data, left_on=['County','State'], right_on=['County','State'],how='right')
merge1[['Number_of_beds','Number_of_hospitals']] = merge1[['Number_of_beds','Number_of_hospitals']].fillna(0)

In [47]:
merge2 = pd.merge(left=merge1, right=bryn_data, left_on=['County','State'], right_on=['County','State'],how='inner')

In [48]:
merged_data = pd.merge(left=merge2, right=nick_data, left_on=['County','State'], right_on=['County','State'],how='inner')

In [49]:
merged_data

,Number_of_beds,Number_of_hospitals,State,County,Urban_Rural_Code,Deaths_COVID,Deaths_total,never,rarely,sometimes,frequently,always,mask_score,total_population,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
0,3667.0,22.0,CA,Alameda,Large central metro,573,"10,908",0.019,0.008,0.055,0.123,0.795,3.667,1671329.0,49.28,11.03,1.06,32.33,0.94,5.35
1,0.0,0.0,CA,Alpine,NaN,0,0,0.025,0.085,0.088,0.190,0.612,3.279,1129.0,67.94,0.35,25.69,1.59,0.00,4.43
2,52.0,1.0,CA,Amador,Noncore,31,415,0.045,0.013,0.099,0.188,0.655,3.395,39752.0,89.65,2.68,2.33,1.67,0.29,3.38
3,553.0,6.0,CA,Butte,Small metro,101,"2,313",0.015,0.043,0.111,0.204,0.626,3.381,219186.0,85.65,1.90,2.53,5.01,0.29,4.62
4,25.0,1.0,CA,Calaveras,Noncore,12,385,0.045,0.019,0.098,0.276,0.562,3.291,45905.0,90.95,1.08,1.97,1.87,0.27,3.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,0.0,WA,Wahkiakum,NaN,0,0,0.045,0.057,0.079,0.161,0.658,3.330,4191.0,93.18,0.43,1.43,0.93,0.19,3.84
129,87.0,1.0,WA,Walla Walla,Small metro,38,651,0.083,0.022,0.061,0.193,0.641,3.287,62201.0,91.44,2.18,1.37,1.70,0.40,2.91
130,253.0,1.0,WA,Whatcom,Small metro,63,"1,881",0.042,0.028,0.061,0.122,0.747,3.504,225300.0,86.46,1.23,3.23,4.48,0.33,4.26
131,50.0,2.0,WA,Whitman,Micropolitan,21,269,0.002,0.009,0.049,0.310,0.629,3.553,50131.0,81.90,1.99,0.84,10.51,0.23,4.52


In [50]:
merged_data.to_csv('Merged_data.csv',header=True)